In [1]:
# 환경 변수 로드
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from dataclasses import dataclass
# 데이터 정의
## 검색어
data = "중국"

## 검색될 내용
@dataclass
class SearchResult:
  keyword: str
  title: str
  link: str
  content: str


In [3]:
# 임시 데이터
datas: list[SearchResult] = []
datas.append(SearchResult(
  data,
  """"중국 손님 안 받아요" 문구에 '발칵'…구청장 나섰다 [소셜픽]""",
  """서울 성동구의 한 카페가 인종차별 논란에 휩싸였습니다.

중국인 손님을 받지 않겠다고 내걸면섭니다.

이 카페의 인스타그램 소개 문구입니다.

영어로 "중국인 손님을 받지 않습니다"라며 우는 얼굴 이모티콘을 달았습니다.

중국에선 이 카페에 갔다가 쫓겨났다는 내용이 퍼지기도 했습니다.

[헨리/영상 크리에이터 : 여기 한국에서 본 카페 중에 가장 인종차별적인 카페예요. 이해가 안 돼요. 왜 내 나라를 이렇게 증오하는지, 누가 이렇게 사업하라고 가르쳤나요?]

소셜미디어를 중심으로 인종차별적이라는 비판이 잇따랐는데, 제재를 해야 한다는 의견에 카페가 위치한 성동구의 자치단체장도 답을 남겼습니다.

정원오 구청장은 "보내주신 우려의 마음을 저 또한 깊이 공감한다"며 "최대한 해당 업장을 설득해볼 수 있도록 노력하겠다"고 밝혔습니다.

이웃나라 일본에서도 비슷한 사건이 있었습니다.

지난해 7월, 도쿄 오오쿠보에 있는 한 레스토랑에서는 "다양성이나 관용으로 말이 많지만 그럴 마음이 없기 때문에 중국인과 한국인은 사절합니다"고 적어 논란이 되기도 했습니다.

[화면출처 인스타그램 'henryli4340']
정재우 앵커 (jung.jaewoo@jtbc.co.kr)""",
  "https://n.news.naver.com/article/437/0000462099?cds=news_media_pc"
))

In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.schema import Document

@dataclass
class Metadata:
  keyword: str
  link: str

contents: list[str] = [("# " + data.title + "\n" + data.content) for data in datas]
metadatas: list[Metadata] = [
  Metadata(
    data.keyword,
    data.link
  ) for data in datas
]

In [ ]:
# 스플리터
# 문서 청크로 나누기
splitter = RecursiveCharacterTextSplitter(
  chunk_size=300,
  chunk_overlap=50
)

docs: list[Document] = []

for i in range(len(contents)):
  splits = splitter.create_documents(
    texts=[contents[i]],
    metadatas=[{
      "keyword":metadatas[i].keyword,
      "link":metadatas[i].link,
    }],
  )
  docs.extend(splits)
len(docs)

2

In [9]:
# vstore 생성
from langchain_upstage import UpstageEmbeddings
from langchain_chroma import Chroma

# 모델 선정
vstore = Chroma(
  collection_name="knowledge_base",
  embedding_function=UpstageEmbeddings(model="embedding-query"),
  persist_directory="./chroma"
)

In [10]:
# vstore에 문서 삽입
vstore.add_documents(docs)

['784a09f9-c1c3-4933-8f64-661fa97d4ae1',
 'c55395bc-0662-482f-9bd8-14de5fb89403']